# 2-Preparation des donnees

La préparation des données, également appelée prétraitement des données ou nettoyage des données, est une étape cruciale du processus d'analyse de données et d'apprentissage automatique. Son but principal est d'améliorer la qualité et la fiabilité des données en éliminant les erreurs, les incohérences et les imprécisions qui pourraient affecter négativement les performances des modèles d'apprentissage automatique ou les résultats d'une analyse de données. Voici quelques objectifs spécifiques de la préparation des données :

- Nettoyage des données : corriger ou supprimer les erreurs, les données manquantes ou les valeurs aberrantes qui pourraient fausser l'analyse.
- Standardisation : harmoniser les données en utilisant des formats et des unités de mesure cohérents.
- Transformation des données : convertir les données brutes en formats plus adaptés pour l'analyse ou l'apprentissage automatique, par exemple en créant de nouvelles variables à partir de variables existantes.
- Intégration : combiner des données provenant de différentes sources pour créer un ensemble de données complet et cohérent.
- Réduction de la dimensionnalité : sélectionner les variables les plus pertinentes pour l'analyse ou l'apprentissage automatique, en éliminant les redondances et les données non pertinentes.

# Imports 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Chargement des données

Nous avons trois datasets :
- `data_x`: les données d'entrée
- `data_y`: les données de sorties, c'est-à-dire la variation du prix de futures d'électricité pour les enregistrements de `data_x`
- `dataNew_x` : les nouvelles données d'entrée a prédire (de meme format et de meme dimensions que data_x)

In [2]:
raw_data_x = pd.read_csv("data/Data_X.csv", index_col='ID')
raw_data_y = pd.read_csv("data/Data_Y.csv", index_col='ID')

# Fusion des ensembles de données

In [3]:
data_xy = pd.merge(raw_data_x, raw_data_y, on='ID')

In [4]:
data_xy.shape

(1494, 35)

Notre ensemble de données est composé de 1494 lignes et de 35 colonnes.

# Verification et Traitement des valeurs manquantes

In [5]:
data_xy.isna().sum().sum()

1002

Il y a 1002 valeurs manquantes.

## Verification des variables

In [6]:
missing_values_cols = data_xy.isna().sum()
missing_values_cols = missing_values_cols[missing_values_cols != 0]
missing_values_cols

DE_FR_EXCHANGE     25
FR_DE_EXCHANGE     25
DE_NET_EXPORT     124
FR_NET_EXPORT      70
DE_NET_IMPORT     124
FR_NET_IMPORT      70
DE_RAIN            94
FR_RAIN            94
DE_WIND            94
FR_WIND            94
DE_TEMP            94
FR_TEMP            94
dtype: int64

On observe que 12 variables possédent des valeurs manquantes.
Au plus il manque 8% (124/1494) valeurs pour une variable.
De plus, on observe des groupes de variables mesuré ensemble, c'est-a-dire, si une de ces variables est manquantes les autres variables du meme groupe seront manquantes aussi.<br> On observe les groupes suivant :
- `DE_FR_EXCHANGE` et `FR_DE_EXCHANGE` : Si l'echange entre la France et l'Allemangne n'est pas mesuré ces deux variables seront manquantes.
- `X_NET_IMPORT` et `X_NET_EXPORT` : Il se peut que pour certain jour les mesures de l'import et l'export ne soit pas disponibles pour un pays en particulier.
-  `FR_RAIN`, `FR_WIND`, `FR_TEMP`, `DE_RAIN`, `DE_WIND`, `DE_TEMP` :  Il se peut que pour certain jour les mesures météorologiques ne soit pas disponibles pour les deux pays.

## Vérification des lignes

In [7]:
missing_values_rows = data_xy.isna().sum(axis=1)
missing_values_rows = missing_values_rows[missing_values_rows != 0]
print(f"Nous avons {len(missing_values_rows)} lignes contenant des valeurs manquantes")

Nous avons 218 lignes contenant des valeurs manquantes


In [8]:
min_missing_values_row = missing_values_rows.min()
max_missing_values_row = missing_values_rows.max()
print("Minimum de valeurs manquantes par ligne : ", min_missing_values_row)
print("Maximum de valeurs manquantes par ligne : ", max_missing_values_row)

Minimum de valeurs manquantes par ligne :  2
Maximum de valeurs manquantes par ligne :  6


### Regardons les variables manquantes dans une ligne contenant un minimum de valeurs manquantes

In [9]:
min_idx = missing_values_rows.idxmin()
min_missing_variables = data_xy.loc[min_idx].isna()
min_missing_variables = min_missing_variables[min_missing_variables == True]
min_missing_variables

DE_NET_EXPORT    True
DE_NET_IMPORT    True
Name: 1054, dtype: bool

En regardant une ligne contenant un minimum (non nulle) de valeurs manquantes, on observe le groupe `DE_NET_EXPORT` et `DE_NET_IMPORT`.

### Regardons les variables manquantes dans une ligne contenant un minimum de valeurs manquantes

In [10]:
max_idx = missing_values_rows.nlargest(2).index
max_missing_variables = data_xy.loc[max_idx[1]].isna()
max_missing_variables = max_missing_variables[max_missing_variables == True]
max_missing_variables

DE_RAIN    True
FR_RAIN    True
DE_WIND    True
FR_WIND    True
DE_TEMP    True
FR_TEMP    True
Name: 1715, dtype: bool

En regardant une ligne contenant un minimum (non nulle) de valeurs manquantes, on observe le groupe des mesures météoroliques pour les deux pays.

<br>
On observe des variables manquantes dans 12 colonnes differentes. Les valeurs manquantes viennet de plusieurs groupes de variables mesurés ensemble.
Les 208 lignes manquantes represente 208/1494 = 14% des lignes de notre ensemble de données. Le nombre de valeurs manquantes par lignes ou par colonnes ne justifie pas la suppresion de celles-ci.
Nous pourrons imputer les valeurs manquantes par la moyenne, la médiane ou supprimer ces valeurs manquantes pour effectuer la modélisation des données.

<br>Pour la suite de la préparation des donées, nous supprimons les données manquantes

In [11]:
# Supression des valeurs manquantes
data_xy.dropna(inplace=True)

# Verification de la comparabilité 

In [12]:

# Calcul des statistiques et recherche du min et du max pour chaque statistique
description = data_xy.drop(columns=['DAY_ID','TARGET']).describe().loc[['mean', 'std', 'min', 'max']]

min_mean = description.loc['mean'].idxmin()
max_mean = description.loc['mean'].idxmax()

min_std = description.loc['std'].idxmin()
max_std = description.loc['std'].idxmax()

min_min = description.loc['min'].idxmin()
max_min = description.loc['min'].idxmax()

min_max = description.loc['max'].idxmin()
max_max = description.loc['max'].idxmax()

# Création de la table Markdown
from IPython.display import Markdown

markdown_table = f"""
|      | Min | Max |
|------|-----|-----|
| Mean | {description.loc['mean', min_mean]:.3f} ({min_mean}) | {description.loc['mean', max_mean]:.3f} ({max_mean}) |
| Std  | {description.loc['std', min_std]:.3f} ({min_std}) | {description.loc['std', max_std]:.3f} ({max_std}) |
| Min  | {description.loc['min', min_min]:.3f} ({min_min}) | {description.loc['min', max_min]:.3f} ({max_min}) |
| Max  | {description.loc['max', min_max]:.3f} ({min_max}) | {description.loc['max', max_max]:.3f} ({max_max}) |
"""

Markdown(markdown_table)


|      | Min | Max |
|------|-----|-----|
| Mean | -0.577 (FR_NUCLEAR) | 0.771 (DE_GAS) |
| Std  | 0.460 (FR_COAL) | 1.186 (FR_WINDPOW) |
| Min  | -5.787 (FR_TEMP) | -0.795 (FR_COAL) |
| Max  | 0.893 (DE_NUCLEAR) | 9.473 (FR_RAIN) |


Nous obeservons que les moyennes des différents variabkes ne sont pas du tout comparables. En effet V1 as une moyenne de 588 et V2 de -0,6. Celà rend les variables V1 et V2 inaptes as leur utilisation dans un modèle de machine learning.
Afin de régler ce problème il faut donc normaliser les variables. La phase de normalisation se déroulera dans les fichiers de création des différents modèles

# Calcul de nouveaux attributs

In [15]:
data_xy['FR_PROD_RENEWABLE'] = data_xy['FR_SOLAR'] + data_xy['FR_HYDRO'] + data_xy['FR_WINDPOW'] + data_xy['FR_NUCLEAR']
data_xy['DE_PROD_RENEWABLE'] = data_xy['DE_SOLAR'] + data_xy['DE_HYDRO'] + data_xy['DE_WINDPOW'] + data_xy['DE_NUCLEAR']

data_xy['DE_PROD_FOSIL'] = data_xy['DE_COAL'] + data_xy['DE_GAS'] + data_xy['DE_LIGNITE']
data_xy['FR_PROD_FOSSIL'] = data_xy['FR_COAL'] + data_xy['FR_GAS']

data_xy['FR_CONS_RENEWABLE'] = data_xy['FR_CONSUMPTION'] - data_xy['FR_RESIDUAL_LOAD']
data_xy['DE_CONS_RENEWABLE'] = data_xy['DE_CONSUMPTION'] - data_xy['DE_RESIDUAL_LOAD']

data_xy['FR_TOTAL_PROD'] = data_xy['FR_GAS'] + data_xy['FR_COAL'] + data_xy['FR_HYDRO'] + data_xy['FR_NUCLEAR'] \
                            + data_xy['FR_SOLAR'] + data_xy['FR_WINDPOW']

data_xy['DE_TOTAL_PROD'] = data_xy['DE_GAS'] + data_xy['DE_COAL'] + data_xy['DE_HYDRO'] + data_xy['DE_NUCLEAR'] \
                            + data_xy['DE_SOLAR'] + data_xy['DE_WINDPOW'] + data_xy['DE_LIGNITE']

# Separation France/ Allemagne

In [16]:
data_fr = data_xy[data_xy['COUNTRY'] == 'FR'].drop(columns=['COUNTRY', 'DAY_ID'])
data_fr.to_csv("data/Data_FR.csv", index=True)
data_fr.shape

(680, 41)

In [17]:
data_de = data_xy[data_xy['COUNTRY'] == 'DE'].drop(columns=['COUNTRY', 'DAY_ID'])
data_de.to_csv("data/Data_DE.csv", index=True)
data_de.shape

(596, 41)